Loading the data

id: household

Choices:  
- heinz41  
- heinz32  
- heinz28  
- hunts32  

variables  
- disp (1/0)  
- feat (1/0)  
- price (Log?)  

In [1]:
import numpy as np
import pandas as pd
df = pd.read_pickle("data.pkl") 
df.head()

,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz28,2
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz28,2
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28,2


Defining the likelihood function using numba and numpy arrays and multiple draws

In [2]:
import numpy as np
import pandas as pd
import datetime
import time
from scipy.optimize import minimize
from numba import jit, prange

@jit(nopython=True, parallel=True)
def likelihood(c, data, draws):
    #print("evaluation likelihood")
    n_r = draws.shape[0]
    n_q = draws.shape[1]
#   n_k = draws.shape[2]
    
    #matrix with all simulations
    simulations = np.zeros((n_q, n_r))
    
    #iterate over households
    for q in prange(n_q):
        rows = data[np.where(data[:,0] == q+1)]
        n_rows = len(rows)
        
        #iterate over draws per household
        for r in prange(n_r):
            probabilities= np.zeros(n_rows)
            
            #iterate over oberservations per househould
            for t in prange(n_rows):
                choices = np.zeros(4)
                
                #itetate over probability of choices per observation
                for j in prange(4):
                    utility = 0; #start with alpha
                    if j < 3: utility = c[j]
                    x = [rows[t][1+j], rows[t][5+j], rows[t][9+j]]
                    mu =  c[3:6]
                    sigma = c[6:]
                    #np.exp(alpha +  np.dot(c[3:6],x) + np.dot(np.multiply(c[6:],draws[row[0]-1]),x))
                    for l in prange(3):
                          utility += mu[l] * x[l] + sigma[l] * draws[r][q][l] * x[l]

                    choices[j] = np.exp(utility)
            
                probabilities[t] = choices[int(rows[t][13])] / np.sum(choices)
                
            simulations[q,r] = np.exp(np.log(probabilities).sum())
            
    estimates = np.zeros(n_q)
    for q in prange(n_q):
        estimates[q] = np.sum(simulations[q,:]) / n_r #.mean()
    return -np.log(estimates).sum()

iterations = 1

   
def mixedlogit(data, drawtype, n_draws, c_0=False, method='BFGS', verbose=False):
    n_q = len(data.id.unique())
    
#     coefficients = [#alpha heinz41
#                     #alpha heinz32 
#                     #alpha hunts32
#                     #mu    display
#                     #mu    feat
#                     #mu    price
#                     #sigma dispay 
#                     #sigma feat 
#                     #sigma price 
#                     ]
    
    #generate inital values if neccesary
    if c_0 == False:
            #genereate random starting coefficients
            c_0 = np.random.rand(9)
    elif len(c_0) != 9:
        raise Exception("Incorrect initial coefficients")
    
    #generate draws
    if drawtype == 'pseudo':
        draws = np.random.randn(n_draws, n_q, 3); 
    else:
         raise Exception("Incorrect Drawtype")
    
    
    global iterations
    iterations = 1
    def logging(xk):
        global iterations
        if(verbose):
            print("Iteration %d, coefficients:" % (iterations))
            print(xk)
            print('\n')
            iterations += 1
    
    start = time.time()
    res =  minimize(likelihood, c_0, args=(data.drop(columns='choice').values, draws), method=method, callback=logging)
    end = time.time()
    duration = end-start 
    
    if verbose: print("Optimization done, time elapsed: %s" % str(datetime.timedelta(seconds=round(duration))))
    
    res['duration'] = duration
    return res

benchmarking and comparing pandas and numba implementation

In [4]:
import time
np.random.seed(1234)
coefficients = [1, #alpha heinz41
                3, #alpha heinz32 
                2, #alpha hunts32
                2, #mu    display
                4, #mu    feat
                8, #mu    price
                3, #sigma dispay 
                1, #sigma feat 
                2, #sigma price 
                ]


print(mixedlogit(df, drawtype="pseudo", n_draws=200, c_0=coefficients, method='BFGS', verbose=True))

Iteration 1, coefficients:
[0.91524745 3.11812771 1.93919206 2.0136927  3.98824327 7.64799244
 2.99747385 0.99646189 2.93327987]


Iteration 2, coefficients:
[0.9238364  1.06250013 3.06249906 1.78674382 3.09160854 6.54840003
 2.57848334 1.05873634 3.00745738]


Iteration 3, coefficients:
[1.73357701 0.11825778 2.28913388 1.31139046 1.73457193 4.95916695
 1.81047478 1.06938876 2.6353987 ]


Iteration 4, coefficients:
[ 1.25259281  0.44643733  2.32633251  0.58300597 -0.25577043  2.30271388
  0.45261397  0.9699713   1.63757721]


Iteration 5, coefficients:
[ 1.10845034  0.68686589  2.35778428  1.17317146 -0.56367492 -1.36389815
 -0.43889424  1.27740495  0.29036205]


Iteration 6, coefficients:
[ 1.19702283  0.7435237   2.43208872  0.96657161 -0.47991171 -1.59700558
 -0.4290565   1.38820783  0.21699708]


Iteration 7, coefficients:
[ 1.64993941  0.98901244  2.77329962  0.91327239 -0.41070756 -1.67719258
 -0.77046941  1.64201372  0.2256274 ]


Iteration 8, coefficients:
[ 1.16185039  0.8823